## API Tiingo - Diario

#### Importar las librerías necesarias

In [1]:
import requests
from decouple import config
import pandas as pd
import psycopg2
import datetime

#### Leer la contraseña y el token desde la variable de entorno

In [2]:
my_host = config('DATABASE_HOST')
port = config('DATABASE_PORT')
password = config('DATABASE_PASSWORD')
database_name = config('DATABASE_NAME')
user = config('DATABASE_USER')
token = config('TIINGO_TOKEN')

#### Obtener datos de Tiingo y mostrarlos en un dataframe

In [4]:
class TiingoData:
    # Inicializar atributos de la clase
    def __init__(self, token):
        self.token = token

    def __call__(self, simbolos):
        precios = []

        for simbolo in simbolos:
            url = f"https://api.tiingo.com/tiingo/daily/{simbolo}/prices"
            parametros = {
                'startDate': '2023-01-01',
                'endDate': datetime.date.today().isoformat(),
                'format': 'json',
            }
            headers = {
                'Authorization': f'Token {self.token}'  # Agregar el token enmascarado en los headers
            }

            try:
                respuesta = requests.get(url, params=parametros, headers=headers)
                respuesta.raise_for_status()  # Lanzar una excepción si la solicitud no es exitosa
                datos = respuesta.json()
                
                # Verificar la estructura de los datos
                if isinstance(datos, list) and all(isinstance(item, dict) for item in datos):
                    for dato in datos:
                        dato['ticker'] = simbolo  # Agregar el símbolo a cada fila porque si no, no lo trae
                    precios.extend(datos)
                else:
                    print(f"Los datos recibidos para el símbolo {simbolo} no están en el formato esperado.")
            except requests.exceptions.RequestException as e:
                print(f"Error al hacer la solicitud para el símbolo {simbolo}:", e)

        if precios:
            # Convertir los datos en un DataFrame de pandas
            df = pd.DataFrame(precios)
            
            # Filtrar las columnas no deseadas
            columnas_no_deseadas = ['adjClose', 'adjHigh', 'adjLow', 'adjOpen', 'adjVolume', 'divCash', 'splitFactor']
            df = df.drop(columns=columnas_no_deseadas, errors='ignore')
            
            # Filtrar y renombrar las columnas necesarias
            columnas = {
                'date': 'Fecha',
                'ticker': 'Símbolo',
                'open': 'Apertura',
                'high': 'Máximo',
                'low': 'Mínimo',
                'close': 'Cierre',
                'volume': 'Volumen',
            }
            df.rename(columns=columnas, inplace=True)
            
            # Convertir la columna de fecha a tipo datetime
            df['Fecha'] = pd.to_datetime(df['Fecha'])
            
            return df
        else:
            return None

    def __str__(self):
        return "TiingoData object"

    def __format__(self, format_spec):
        return "TiingoData object"  


# Lista de símbolos a consultar
simbolos = ['AMZN', 'AAPL', 'GOOGL', 'MSFT', 'TSLA']

# Crear instancia de TiingoData
tiingo_data = TiingoData(token)

# Obtener y mostrar los datos
datos_tiingo_df = tiingo_data(simbolos)
if datos_tiingo_df is not None:
    print(datos_tiingo_df)

print(tiingo_data)  # Prueba de __str__
print(f"{tiingo_data:custom_format}")  # Prueba de __format__


                         Fecha  Cierre  Máximo   Mínimo  Apertura    Volumen  \
0    2023-01-03 00:00:00+00:00   85.82   86.96   84.205     85.46   76706040   
1    2023-01-04 00:00:00+00:00   85.14   86.98   83.360     86.55   68885123   
2    2023-01-05 00:00:00+00:00   83.12   85.42   83.070     85.33   67930825   
3    2023-01-06 00:00:00+00:00   86.08   86.40   81.430     83.03   83303361   
4    2023-01-09 00:00:00+00:00   87.36   89.48   87.080     87.46   65266056   
...                        ...     ...     ...      ...       ...        ...   
1550 2024-03-22 00:00:00+00:00  170.83  171.20  166.300    166.69   75580637   
1551 2024-03-25 00:00:00+00:00  172.63  175.24  168.730    168.76   74228615   
1552 2024-03-26 00:00:00+00:00  177.67  184.25  177.380    178.58  113186227   
1553 2024-03-27 00:00:00+00:00  179.83  181.91  176.000    181.41   81804043   
1554 2024-03-28 00:00:00+00:00  175.79  179.57  175.300    177.45   77654838   

     Símbolo  
0       AMZN  
1       A

#### Conectarse con Amazon Redshift y crear tabla (si no existe)

In [55]:
from datetime import datetime

# Fecha y hora actual
fecha_actual = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

#Conectar con la base de datos
try:
    conn = psycopg2.connect(
        host=my_host, 
        port=port, 
        database=database_name, 
        user=user, 
        password=password
    )
    print("Conexión a la base de datos exitosa.")

    # Crear tabla con columna temporal de fecha y hora de ingesta
    try:
        cur = conn.cursor()
        cur.execute("""
        CREATE TABLE IF NOT EXISTS hvzambrana_coderhouse.tiingo (
            simbolo VARCHAR(10) DISTKEY NOT NULL,
            fecha_ingesta TIMESTAMP DEFAULT CURRENT_TIMESTAMP, 
            fecha TIMESTAMP SORTKEY NOT NULL,
            apertura FLOAT(15) NULL,
            maximo FLOAT(15) NULL,
            minimo FLOAT(15) NULL,
            cierre FLOAT(15) NULL,
            volumen INTEGER NULL
        );
        """)
        conn.commit()
        print("Tabla creada exitosamente.")
    except Exception as e:
            print(f"Error al crear la tabla:", e) # Imprimir la descripción del error

except Exception as e:
    print(f"Error al crear la conexión a la base de datos:", e) # Imprimir la descripción del error
finally:
    if conn is not None:
        conn.close()
        print("Conexión cerrada correctamente.")

Conexión a la base de datos exitosa.
Tabla creada exitosamente.
Conexión cerrada correctamente.
